In [1]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import Model, Sequential

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError

from tensorflow.keras.layers import Dense, Conv1D, LSTM, Lambda, Reshape, RNN, LSTMCell,ConvLSTM2D

import warnings
warnings.filterwarnings('ignore')

In [2]:
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['axes.grid'] = False

In [3]:
print(tf.__version__)

2.12.0


In [4]:
tf.random.set_seed(42)
np.random.seed(42)

In [5]:
df = pd.read_csv('../../FinBERT/Data_final/BTC_final.csv').drop(['CionScore','CoinDate'],axis=1)
df = df.drop(['Unnamed: 0'], axis=1)
df.head()

,Open,High,Low,Close,Volume
0,387.427002,391.378998,380.779999,383.614990,26229400.0
1,383.988007,385.497009,372.946014,375.071991,21777700.0
2,375.181000,377.695007,357.859009,359.511993,30901200.0
3,359.891998,364.487000,325.885986,328.865997,47236500.0
4,328.915985,341.800995,289.295990,320.510010,83308096.0


In [6]:
df.tail()

,Open,High,Low,Close,Volume
3068,23946.00781,24103.70508,23007.07227,23198.12695,2.681174e+10
3069,23200.12500,23210.21094,22861.55859,23175.37500,1.610072e+10
3070,23174.15039,23654.36719,23084.22070,23561.21289,1.664453e+10
3071,23561.45117,23857.89063,23205.87891,23522.87109,2.266076e+10
3072,23512.17773,23521.54688,23400.39648,23433.81641,2.219076e+10


In [7]:
df.shape

(3073, 5)

In [8]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Open,3073.0,1.308273e+04,1.609732e+04,1.768970e+02,7.155550e+02,7.296165e+03,1.781364e+04,6.754973e+04
High,3073.0,1.340983e+04,1.650774e+04,2.117310e+02,7.364520e+02,7.447272e+03,1.839395e+04,6.878962e+04
Low,3073.0,1.272014e+04,1.562394e+04,1.715100e+02,7.052600e+02,7.153306e+03,1.738255e+04,6.638206e+04
Close,3073.0,1.308870e+04,1.609327e+04,1.781030e+02,7.164110e+02,7.302089e+03,1.789970e+04,6.756683e+04
Volume,3073.0,1.656535e+10,1.985180e+10,5.914570e+06,1.123540e+08,8.660880e+09,2.806636e+10,3.509680e+11


In [9]:
# df['Coin_timestamp'] = timestamp_s = pd.to_datetime(df['CoinDate']).map(datetime.datetime.timestamp)

In [10]:
# df = df.drop(['CoinDate'], axis=1)
df.tail()

,Open,High,Low,Close,Volume
3068,23946.00781,24103.70508,23007.07227,23198.12695,2.681174e+10
3069,23200.12500,23210.21094,22861.55859,23175.37500,1.610072e+10
3070,23174.15039,23654.36719,23084.22070,23561.21289,1.664453e+10
3071,23561.45117,23857.89063,23205.87891,23522.87109,2.266076e+10
3072,23512.17773,23521.54688,23400.39648,23433.81641,2.219076e+10


In [11]:
n = len(df)

# Split 70:20:10 (train:validation:test)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

train_df.shape, val_df.shape, test_df.shape

((2151, 5), (614, 5), (308, 5))

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_df)

train_df[train_df.columns] = scaler.transform(train_df[train_df.columns])
val_df[val_df.columns] = scaler.transform(val_df[val_df.columns])
test_df[test_df.columns] = scaler.transform(test_df[test_df.columns])

In [13]:
train_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Open,2151.0,0.215505,0.212028,0.0,0.013279,0.177392,0.395715,1.0
High,2151.0,0.213083,0.211984,0.0,0.011472,0.173223,0.394385,1.0
Low,2151.0,0.215220,0.210212,0.0,0.013610,0.179136,0.394656,1.0
Close,2151.0,0.215487,0.211927,0.0,0.013218,0.177176,0.395720,1.0
Volume,2151.0,0.104343,0.155503,0.0,0.000735,0.025612,0.165412,1.0


In [14]:
train_df.to_csv('data/train.csv')
val_df.to_csv('data/val.csv')
test_df.to_csv('data/test.csv')

In [22]:
# train_df = pd.read_csv('../data/train.csv', index_col=0)
# val_df = pd.read_csv('../data/val.csv', index_col=0)
# test_df = pd.read_csv('../data/test.csv', index_col=0)

print(train_df.shape, val_df.shape, test_df.shape)

(2151, 5) (614, 5) (308, 5)


In [15]:
class DataWindow():
    def __init__(self, input_width, label_width, shift, 
                 train_df=train_df, val_df=val_df, test_df=test_df, 
                 label_columns=None):
        
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}
        
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift
        
        self.total_window_size = input_width + shift
        
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
        
        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
    
    def split_to_inputs_labels(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:,:,self.column_indices[name]] for name in self.label_columns],
                axis=-1
            )
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])
        
        return inputs, labels
    
    def plot(self, model=None, plot_col='Close', max_subplots=3):
        inputs, labels = self.sample_batch
        
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        
        for n in range(max_n):
            plt.subplot(3, 1, n+1)
            plt.ylabel(f'{plot_col} [scaled]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                     label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
              label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
              label_col_index = plot_col_index

            if label_col_index is None:
              continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', marker='s', label='Labels', c='green', s=64)
            if model is not None:
              predictions = model(inputs)
              plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                          marker='X', edgecolors='k', label='Predictions',
                          c='red', s=64)

            if n == 0:
              plt.legend()

        plt.xlabel('Time (h)')
        
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32
        )
        
        ds = ds.map(self.split_to_inputs_labels)
        return ds
    
    @property
    def train(self):
        return self.make_dataset(self.train_df)
    
    @property
    def val(self):
        return self.make_dataset(self.val_df)
    
    @property
    def test(self):
        return self.make_dataset(self.test_df)
    
    @property
    def sample_batch(self):
        result = getattr(self, '_sample_batch', None)
        if result is None:
            result = next(iter(self.train))
            self._sample_batch = result
        return result

In [16]:
def compile_and_fit(model, window, patience=3, max_epochs=50):
    early_stopping = EarlyStopping(monitor='val_loss',
                                   patience=patience,
                                   mode='min')
    
    model.compile(loss=MeanSquaredError(),
                  optimizer=Adam(),
                  metrics=[MeanAbsoluteError()])
    
    history = model.fit(window.train,
                       epochs=max_epochs,
                       validation_data=window.val,
                       callbacks=[early_stopping])
    
    return history

In [17]:
column_indices = {name: i for i, name in enumerate(train_df.columns)}

## CNN 

### Multi-step model 

In [18]:
multi_window = DataWindow(input_width=7, label_width=7, shift=1, label_columns=['Close'])

In [19]:
ms_mae_val = []
ms_mae_test = []

In [20]:
ms_lstm_model = Sequential([
    # LSTM(64, return_sequences=True),
    Dense(units=1,kernel_initializer=tf.initializers.zeros),
])

history = compile_and_fit(ms_lstm_model, multi_window)

ms_val_performance = {}
ms_performance = {}

ms_val_performance['LSTM'] = ms_lstm_model.evaluate(multi_window.val)
ms_performance['LSTM'] = ms_lstm_model.evaluate(multi_window.test, verbose=0)

print(f"驗證集的MSE {ms_val_performance['LSTM'][1]}")
print(f"測試集的MSE {ms_performance['LSTM'][1]}")

Epoch 1/50
67/67 [==============================] - 1s 4ms/step - loss: 0.0589 - mean_absolute_error: 0.1725 - val_loss: 2.4677 - val_mean_absolute_error: 1.4545
Epoch 2/50
67/67 [==============================] - 0s 2ms/step - loss: 0.0220 - mean_absolute_error: 0.1175 - val_loss: 1.3020 - val_mean_absolute_error: 1.0453
Epoch 3/50
67/67 [==============================] - 0s 2ms/step - loss: 0.0103 - mean_absolute_error: 0.0870 - val_loss: 0.7643 - val_mean_absolute_error: 0.7925
Epoch 4/50
67/67 [==============================] - 0s 2ms/step - loss: 0.0065 - mean_absolute_error: 0.0702 - val_loss: 0.4966 - val_mean_absolute_error: 0.6331
Epoch 5/50
67/67 [==============================] - 0s 2ms/step - loss: 0.0045 - mean_absolute_error: 0.0580 - val_loss: 0.3392 - val_mean_absolute_error: 0.5191
Epoch 6/50
67/67 [==============================] - 0s 2ms/step - loss: 0.0031 - mean_absolute_error: 0.0476 - val_loss: 0.2359 - val_mean_absolute_error: 0.4297
Epoch 7/50
67/67 [==========